## MAL API Setup

In [1]:
import requests

api_url = 'https://api.myanimelist.net/v2'

# A Client ID is needed (https://myanimelist.net/apiconfig)
#with open('client_id.txt', 'r') as f:
#    CLIENT_ID = f.read()

headers = {'X-MAL-CLIENT-ID': "ec1c4b7aa5f59e854390b2259667340c"}

def get_data(endpoint, params=None):
    url = api_url + endpoint
    if params:
        url += '?' + '&'.join(f'{key}={value}' for key, value in params.items())
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()


## Scrap Anime

In [2]:
import json
import os

scraping_save_pages = '../My Anime List Scrapping/data/data_tmp/anime_pages'

if not os.path.exists(scraping_save_pages): # Create saving directory if it doesn't exist
  os.makedirs(scraping_save_pages)

endpoint = f'/anime/ranking'
limit = 500

anime_keys = ['id', 'title', 'main_picture', 'alternative_titles', 'start_date', 'end_date', 'synopsis', 'mean', 'rank', 'popularity',
              'num_list_users', 'num_scoring_users', 'num_favorites', 'nsfw', 'genres', 'created_at', 'updated_at', 'media_type', 'status',
              'num_episodes', 'start_season', 'broadcast', 'source', 'average_episode_duration', 'rating','studios']

def scrape_page(page):
    params = {'ranking_type': 'bypopularity', 'limit': limit, 'offset': page*limit, 'fields': ','.join(anime_keys)}
    data = get_data(endpoint, params)
    useful = [anime['node'] for anime in data['data']]
    with open(scraping_save_pages + f'/page{str(page).zfill(2)}.json', 'w') as f:
        json.dump(useful, f, indent=4)

In [3]:
import math

# 12  OCT 2022
previous_total_anime = 29_800
previous_last_page = math.ceil(previous_total_anime / limit) - 1

data = get_data(endpoint, {'ranking_type': 'bypopularity', 'limit': limit, 'offset': previous_last_page*limit, 'fields': ','.join(anime_keys)})
#data = get_data(endpoint,{'ranking_type': 'bypopularity', 'limit': limit, 'offset': previous_last_page*limit, 'fields': ','.join(str(v) for v in 'recommendations')})
#assert len(data['data']) > 0
assert 'next' not in data['paging']


last_page = previous_last_page

last_page

59

In [4]:
import tqdm
import time

for page in tqdm.trange(last_page+1):
    scrape_page(page)
    time.sleep(1)

100%|██████████| 60/60 [02:54<00:00,  2.91s/it]


## Merge Files

In [5]:
import os

scraping_save_pages = 'data/data_tmp/anime_pages'

data = []
for file_name in os.listdir(scraping_save_pages):
    file_path = os.path.join(scraping_save_pages, file_name)
    with open(file_path, 'r') as f:
        file = json.load(f)
    data.extend(file)

len(data),type(data)

(23936, list)

In [6]:
len(data),len(data[1])
data[1]

{'id': 1535,
 'title': 'Death Note',
 'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/9/9453.jpg',
  'large': 'https://api-cdn.myanimelist.net/images/anime/9/9453l.jpg'},
 'alternative_titles': {'synonyms': ['DN'], 'en': 'Death Note', 'ja': 'デスノート'},
 'start_date': '2006-10-04',
 'end_date': '2007-06-27',
 'synopsis': 'Brutal murders, petty thefts, and senseless violence pollute the human world. In contrast, the realm of death gods is a humdrum, unchanging gambling den. The ingenious 17-year-old Japanese student Light Yagami and sadistic god of death Ryuk share one belief: their worlds are rotten.\n\nFor his own amusement, Ryuk drops his "Death Note" into the human world. Light stumbles upon it, deeming the first of its rules ridiculous: the human whose name is written in this note shall die. However, the temptation is too great, and Light experiments by writing a felon\'s name, which disturbingly enacts his first murder.\n\nAware of the terrifying godlike powe

In [7]:
with open('data/data_tmp/anime_raw.json', 'w') as f:
    json.dump(data, f, indent=4)

## Anime Cleaning

In [8]:

import pandas as pd
import numpy as np
import json

with open('data/data_tmp/anime_raw.json', 'r') as f:
    data = json.load(f)

anime = pd.json_normalize(data, sep='_')

# Use Timestamps
anime['start_date'] = pd.to_datetime(anime['start_date'])
anime['end_date'] = pd.to_datetime(anime['end_date'])

# Avoid floats and zeroes marking nsfw
anime['num_episodes'] = anime['num_episodes'].replace(0, np.nan).astype('Int64')
anime['popularity'] = anime['popularity'].replace(0, np.nan).astype('Int64')
anime['rank'] = anime['rank'].replace(0, np.nan).astype('Int64')
anime['mean'] = anime['mean'].replace(0, np.nan).astype('float64')
anime['num_favorites'] = anime['num_favorites'].replace(0, np.nan).astype('Int64')

# Use Timedelta
anime['average_episode_duration'] = pd.to_timedelta(anime['average_episode_duration'].replace(0, np.nan), unit='s')

# Avoid floats, as time
anime['start_season_year'] = anime['start_season_year'].astype('Int64')
anime['broadcast_start_time'] = pd.to_datetime(anime['broadcast_start_time']).dt.time

# Only keep names
anime['genres'] = anime['genres'].apply(lambda x: [dic['name'] for dic in x] if not x is np.nan else [])
#anime['recommendations'] = anime['recommendations'].apply(lambda x: [dic['title'] for dic in x] if not x is np.nan else [])
anime['studios'] = anime['studios'].apply(lambda x: [dic['name'] for dic in x] if not x is np.nan else [])
#anime['recommendations'] = anime['recommendations'].apply(lambda x: [dic['name'] for dic in x] if not x is np.nan else [])
# MyAnimeList edits
anime['created_at'] = pd.to_datetime(anime['created_at']).dt.tz_convert(None)
anime['updated_at'] = pd.to_datetime(anime['updated_at']).dt.tz_convert(None)

# Avoid empty string
anime['synopsis'] = anime['synopsis'].replace('', np.nan)
#anime['recommendations'] = anime['recommendations'].replace('', np.nan)
anime['alternative_titles_en'] = anime['alternative_titles_en'].replace('', np.nan)
anime['alternative_titles_ja'] = anime['alternative_titles_ja'].replace('', np.nan)
                

order = ['id', 'title', 'media_type', 'mean', 'num_scoring_users',                          # 10 Most important attributes, 
        'status', 'num_episodes', 'start_date', 'end_date', 'source',                      # appearing first on kaggle

        'num_list_users', 'popularity', 'num_favorites', 'rank',                           # Other important
        'average_episode_duration', 'rating', 'start_season_year',                         # attributes
        'start_season_season', 'broadcast_day_of_the_week', 'broadcast_start_time',   

        'genres', 'studios',                                                               # Multivalued attributes
        'synopsis', 'nsfw', 'created_at', 'updated_at',                                  # Description, MyAnimeList edits
        
        'main_picture_medium', 'main_picture_large',                                       # Media data
        'alternative_titles_en', 'alternative_titles_ja', 'alternative_titles_synonyms']   # Other titles


anime = anime[order]

# Save to csv
anime.to_csv('data/anime_Feb23.csv', index=False)

pd.options.display.max_columns = None
anime.head(1)

,id,title,media_type,mean,num_scoring_users,status,num_episodes,start_date,end_date,source,num_list_users,popularity,num_favorites,rank,average_episode_duration,rating,start_season_year,start_season_season,broadcast_day_of_the_week,broadcast_start_time,genres,studios,synopsis,nsfw,created_at,updated_at,main_picture_medium,main_picture_large,alternative_titles_en,alternative_titles_ja,alternative_titles_synonyms
0,16498,Shingeki no Kyojin,tv,8.53,2585993,finished_airing,25,2013-04-07,2013-09-29,manga,3635996,1,159231,109,0 days 00:24:00,r,2013,spring,sunday,01:58:00,"[Action, Award Winning, Drama, Gore, Military,...",[Wit Studio],"Centuries ago, mankind was slaughtered to near...",white,2012-12-05 12:03:21,2023-01-12 15:12:07,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,Attack on Titan,進撃の巨人,"[AoT, SnK]"


## Load Anime

In [10]:
from ast import literal_eval
import pandas as pd

data= pd.read_csv('data/anime_Feb23.csv')

columns_dtype_datetime = ['start_date', 'end_date', 'created_at', 'updated_at']
for col in columns_dtype_datetime:
    data[col] = pd.to_datetime(data[col])

columns_dtype_Int64 = ['num_episodes', 'popularity', 'rank', 'start_season_year']
for col in columns_dtype_Int64:
    data[col] = data[col].astype('Int64')

columns_dtype_list = ['genres', 'studios', 'alternative_titles_synonyms']
for col in columns_dtype_list:
    data[col] = data[col].apply(literal_eval)

data['broadcast_start_time'] = pd.to_datetime(data['broadcast_start_time']).dt.time   # Time of day

data['average_episode_duration'] = pd.to_timedelta(data['average_episode_duration'])  # Duration

data.head(1)

,id,title,media_type,mean,num_scoring_users,status,num_episodes,start_date,end_date,source,num_list_users,popularity,num_favorites,rank,average_episode_duration,rating,start_season_year,start_season_season,broadcast_day_of_the_week,broadcast_start_time,genres,studios,synopsis,nsfw,created_at,updated_at,main_picture_medium,main_picture_large,alternative_titles_en,alternative_titles_ja,alternative_titles_synonyms
0,16498,Shingeki no Kyojin,tv,8.53,2585993,finished_airing,25,2013-04-07,2013-09-29,manga,3635996,1,159231.0,109,0 days 00:24:00,r,2013,spring,sunday,01:58:00,"[Action, Award Winning, Drama, Gore, Military,...",[Wit Studio],"Centuries ago, mankind was slaughtered to near...",white,2012-12-05 12:03:21,2023-01-12 15:12:07,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,Attack on Titan,進撃の巨人,"[AoT, SnK]"


In [22]:
del data